In [1]:
import pandas as pd
import numpy as np
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.ldamodel import LdaModel
from gensim import corpora
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from google.colab import drive


ModuleNotFoundError: No module named 'gensim'

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

# Mount Google Drive
drive.mount('/content/drive')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Step 2: Load dataset from Google Drive
dataset_file_id = '1WRapeHOrneSf8zq7GtA8M_XscQT_l4VY'  # Replace with your dataset file ID
!gdown --id {dataset_file_id} -O origo_to_lda.xlsx

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1WRapeHOrneSf8zq7GtA8M_XscQT_l4VY
From (redirected): https://drive.google.com/uc?id=1WRapeHOrneSf8zq7GtA8M_XscQT_l4VY&confirm=t&uuid=d9df0503-04a6-498c-ae2c-ee077998a781
To: /content/origo_to_lda.xlsx
100% 264M/264M [00:15<00:00, 17.4MB/s]


In [ ]:
# Step 2: Download custom stopwords from Google Drive
custom_stopwords_file_id = '14jq7yAVWklghA4pKac0sKwDHxWXB_tUK'  # Replace with your custom stopwords file ID
!gdown --id {custom_stopwords_file_id} -O custom_stopwords.txt

# Read the custom stopwords file, trying different encodings
try:
    with open('custom_stopwords.txt', 'r', encoding='utf-8') as file:
        custom_stopwords = file.read().splitlines()
except UnicodeDecodeError:
    try:
        with open('custom_stopwords.txt', 'r', encoding='latin-1') as file:
            custom_stopwords = file.read().splitlines()
    except UnicodeDecodeError:
        with open('custom_stopwords.txt', 'r', encoding='iso-8859-1') as file:
            custom_stopwords = file.read().splitlines()

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=14jq7yAVWklghA4pKac0sKwDHxWXB_tUK
To: /content/custom_stopwords.txt
100% 6.71k/6.71k [00:00<00:00, 30.2MB/s]


In [ ]:
# Load the dataset
df = pd.read_excel('origo_to_lda.xlsx')

# Filter out rows with NA in the text column
df = df.dropna(subset=['text'])

In [ ]:
# Preprocess the text data
def preprocess(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('hungarian')]
    tokens = [word for word in tokens if word not in custom_stopwords]
    tokens = [word for word in tokens if word.strip()]
    return ' '.join(tokens)

df['processed_text'] = df['text'].apply(preprocess)

# Save the DataFrame to CSV in Google Drive
file_path = '/content/drive/My Drive/origo_processed_text.csv'
df.to_csv(file_path, index=False)


In [ ]:
# Create a document-term matrix with TF weighting
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform(df['processed_text'])

# Define the number of topics
k = 30

In [ ]:
# Perform LDA topic modeling
# Create a list of tuples (token_id, token)
corpus_with_tokens = [(vectorizer.vocabulary_.get(w), w) for w in vectorizer.get_feature_names_out()]

# Create the dictionary
# Pass a list of token lists to the Dictionary
id2word = corpora.Dictionary([[token] for _, token in corpus_with_tokens]) # Wrap each token in a list

# Convert documents to bag-of-words format
corpus = [id2word.doc2bow(text.split()) for text in df['processed_text']]

lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=k, random_state=100, update_every=1, passes=10, alpha='auto', per_word_topics=True)
# Define the path where you want to save the model
model_path = '/content/drive/My Drive'

# Save the LDA model
lda_model.save(model_path)


NameError: name 'vectorizer' is not defined

In [ ]:
from gensim.models.ldamodel import LdaModel
model_path = '/content/drive/My Drive/lda_model' # Replace 'your_model_file.model' with the actual filename
# Load the model
lda_model = LdaModel.load(model_path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/lda_model'

In [ ]:
 # Import necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora
from gensim.models import LdaModel
import numpy as np

# Assuming df, vectorizer, and k are already defined...

# Create a list of tuples (token_id, token)
corpus_with_tokens = [(vectorizer.vocabulary_.get(w), w) for w in vectorizer.get_feature_names_out()]

# Create the dictionary
# Pass a list of token lists to the Dictionary
id2word = corpora.Dictionary([[token] for _, token in corpus_with_tokens]) # Wrap each token in a list

# Convert documents to bag-of-words format
corpus = [id2word.doc2bow(text.split()) for text in df['processed_text']]

# Perform LDA topic modeling
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=k, random_state=100, update_every=1, passes=10, alpha='auto', per_word_topics=True)

# Extract topic probabilities
topics = lda_model.get_document_topics(corpus, minimum_probability=0) # Add minimum_probability parameter
topic_probabilities = np.zeros((len(df), k))

for i, doc in enumerate(topics):
    for topic, prob in doc:
        topic_probabilities[i, topic] = prob

In [ ]:
# Convert documents to bag-of-words format
corpus = [id2word.doc2bow(text.split()) for text in df['processed_text']]

# Perform LDA topic modeling
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=k, random_state=100, update_every=1, passes=10, alpha='auto', per_word_topics=True)

# Extract topic probabilities
topics = lda_model.get_document_topics(corpus, minimum_probability=0) # Add minimum_probability parameter
topic_probabilities = np.zeros((len(df), k))

for i, doc in enumerate(topics):
    for topic, prob in doc:
        topic_probabilities[i, topic] = prob

KeyboardInterrupt: 

In [ ]:
# Assign column names for topic probabilities for clarity
topic_prob_df = pd.DataFrame(topic_probabilities, columns=[f'Topic_{i+1}' for i in range(k)])

# Add document IDs from the original dataframe to the topic probabilities
topic_prob_df['id'] = df['id']

In [ ]:
# Calculate the most probable topic for each document
topic_prob_df['most_probable_topic'] = topic_prob_df.iloc[:, :k].idxmax(axis=1)

# Combine the original ID with the most probable topic and probabilities
result_df = pd.concat([df['id'], topic_prob_df], axis=1)

In [ ]:
# Save the complete results to CSV in Google Drive
result_csv_path = '/content/drive/MyDrive/complete_topic_probabilities_origo_lda_30_20240814.csv'
result_df.to_csv(result_csv_path, index=False)

In [ ]:
# Get the most significant terms for each topic
significant_terms = {i: [word for word, _ in lda_model.show_topic(i, topn=20)] for i in range(k)}

# Create a DataFrame for significant terms
significant_terms_df = pd.DataFrame(dict([(f'Topic_{k+1}', pd.Series(v)) for k, v in significant_terms.items()]))

# Save significant terms to CSV in Google Drive
significant_terms_csv_path = '/content/drive/MyDrive/significant_terms_origo_lda30_20240814.csv'
significant_terms_df.to_csv(significant_terms_csv_path, index=False)

# Verify the saved files
print("Topic Probabilities CSV:\n", result_df.head())
print("Significant Terms CSV:\n", significant_terms_df.head())